In [1]:
import os
import re
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

In [2]:
from keras.models import load_model

In [3]:
df_a = pd.read_csv("./df_y4s2_final_new.csv")

In [4]:
df_a = df_a.dropna(subset='mentioned_location')

In [5]:
df_a['relevant'].value_counts()

False    1536
True      271
Name: relevant, dtype: int64

In [6]:
num_drop = int(df_a[df_a['relevant'] == False].shape[0]) - int(df_a[df_a['relevant'] == True].shape[0])
np.random.seed(int(time.time()))
false_rows = df_a[df_a['relevant'] == False]
rows_to_keep = false_rows.sample(n = 271, random_state = 42)

df_dropped = df_a[df_a['relevant'] != False]
df_a = pd.concat([df_dropped,rows_to_keep])
df_a['relevant'].value_counts()

True     271
False    271
Name: relevant, dtype: int64

In [7]:
df_a['word_length'] = df_a['translated_text'].str.len()

In [8]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mrbvb\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# sent_tokenize(data)
# nltk.download('punkt')

for index, row in df_a.iterrows():
    text = row['translated_text']
    words = text.split()  # Split the text into words
    filtered_words = [word for word in words if word.lower() not in stop_words]  # Remove stopwords
    processed_text = " ".join(filtered_words)  # Join the remaining words back into a single string
    df_a.at[index, 'processed_text'] = processed_text  # Store the processed text in a new column

In [10]:

# Assuming df_a is your DataFrame and 'translated_text' is the column name
stop_words = set(stopwords.words("english"))
stop_words.add('http')


# Iterate over each row and process the text
for index, row in df_a.iterrows():
    text = row['translated_text']
    words = text.split()  # Split the text into words
    filtered_words = [word for word in words if word.lower() not in stop_words]  # Remove stopwords
    processed_text = " ".join(filtered_words)  # Join the remaining words back into a single string
    df_a.at[index, 'processed_text'] = processed_text  # Store the processed text in a new column

# Now df_a contains a new column 'processed_text' with the processed text

from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
def stem_words(text):
    return " ".join([stemmer.stem(word) for word in text.split()])
df_a["processed_text"] = df_a["processed_text"].apply(lambda x: stem_words(x))

print(df_a['processed_text'])

0       use walk bt wat phra si laksi red line. distan...
1       walk central ladprao phahon yothin 34, bad. we...
2       use walk huai khwang sutthisan. summary, walk ...
3       korea, walk hard, matter much spent, gain weig...
4       banthat thong would worth walk around -make ne...
                              ...                        
924     best thailand!!!!!!!! good sidewalk, demolishe...
1664    sidewalk terribl condit weather terrible. actu...
1211            usual seat footpath front suea pa stadium
559     #pleas share. hawker watthana district humble,...
405     rt @chocolixirmania silom, saladaeng, flood fo...
Name: processed_text, Length: 542, dtype: object


In [11]:
df_a['word_length'] = df_a['processed_text'].str.len()
df_a['word_length'].sort_values(ascending=False)

2775    462
3584    386
3134    356
1664    354
3443    347
       ... 
118      37
182      34
286      34
562      31
431      27
Name: word_length, Length: 542, dtype: int64

In [12]:
df_a['processed_text']

0       use walk bt wat phra si laksi red line. distan...
1       walk central ladprao phahon yothin 34, bad. we...
2       use walk huai khwang sutthisan. summary, walk ...
3       korea, walk hard, matter much spent, gain weig...
4       banthat thong would worth walk around -make ne...
                              ...                        
924     best thailand!!!!!!!! good sidewalk, demolishe...
1664    sidewalk terribl condit weather terrible. actu...
1211            usual seat footpath front suea pa stadium
559     #pleas share. hawker watthana district humble,...
405     rt @chocolixirmania silom, saladaeng, flood fo...
Name: processed_text, Length: 542, dtype: object

In [13]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from sklearn.preprocessing import LabelEncoder
from keras.layers import Embedding, SpatialDropout1D, Dense, Bidirectional, Flatten, LSTM
from keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K
from keras.optimizers import Adam

MAX_WORDS = 3500  # Memorized words
MAX_SEQUENCE_LENGTH = 132
EMBEDDING_DIM = 100
epochs = 50
batch_size = 32

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=MAX_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~')
tokenizer.fit_on_texts(df_a.processed_text.values)
#word_index = tokenizer.word_index
X = tokenizer.texts_to_sequences(df_a['processed_text'].values)
X = tf.keras.utils.pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
X = X.astype(np.float32)
#Y = df_a['relevant'].values
label_encoder = LabelEncoder()
Y = label_encoder.fit_transform(df_a['relevant'].values)
Y = Y.astype(np.float32)
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20, random_state = 42)

In [14]:
X

array([[   0.,    0.,    0., ..., 1756.,  368.,  607.],
       [   0.,    0.,    0., ...,   40., 1757.,  209.],
       [   0.,    0.,    0., ...,  323., 1759.,  173.],
       ...,
       [   0.,    0.,    0., ...,   21., 1088., 1488.],
       [   0.,    0.,    0., ...,   19., 1023., 1024.],
       [   0.,    0.,    0., ...,   92.,   97., 1176.]], dtype=float32)

In [15]:
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

In [16]:
X_train

array([[0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+0

In [17]:
X_train

array([[0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+0

In [18]:
def recall_m(y_true, y_pred):
    y_true = K.cast(y_true, K.floatx())  # Cast y_true to the same data type as y_pred
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    y_true = K.cast(y_true, K.floatx())  # Cast y_true to the same data type as y_pred
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

In [19]:
X_train.shape

(433, 132)

In [20]:
Y_train.shape

(433,)

In [21]:
import pandas as pd
import math
import keras
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import save_model
from tensorflow.keras.models import model_from_json
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from keras_tuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
from pickle import dump,load
import warnings
warnings.simplefilter("ignore", UserWarning)

C:\Users\mrbvb\AppData\Local\Temp\ipykernel_32876\44952720.py:13: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.engine.hyperparameters import HyperParameters


In [22]:
def build_model(hp):
    model = Sequential()
    model.add(LSTM(hp.Int('input_unit', min_value=32, max_value=512, step=32), return_sequences=True, input_shape=(X_train.shape[1], 1)))
    for i in range(hp.Int('n_layers', 1, 4)):
        model.add(LSTM(hp.Int(f'lstm_{i}_units', min_value=32, max_value=512, step=32), return_sequences=True))
    model.add(LSTM(hp.Int('layer_2_neurons', min_value=32, max_value=512, step=32)))
    model.add(Dropout(hp.Float('Dropout_rate', min_value=0, max_value=0.5, step=0.1)))
    model.add(Dense(1, activation='sigmoid'))  # For binary classification
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['f1'])
    return model


In [23]:
from kerastuner import Objective
from kerastuner.tuners import GridSearch

tuner = GridSearch(
    build_model,
    objective=Objective('f1', direction='max'),
    max_trials=2,
    directory='grid_search',
    project_name='my_grid_search'
)

tuner.search(
    x=X_train,
    y=Y_train,
    epochs=20,
    batch_size=64,
    validation_data=(X_test, Y_test)
)

Trial 2 Complete [00h 00m 01s]

Best f1 So Far: None
Total elapsed time: 00h 00m 04s


# Grid Search

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier

# Function to create LSTM model
def create_lstm_model(units=100, optimizer='adam'):
    model = Sequential()
    model.add(LSTM(units=units, input_shape=(X_train.shape[0], X_train.shape[1])))
    model.add(Dropout(0.2))  # Add dropout layer directly
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Create KerasClassifier for use in scikit-learn GridSearchCV
lstm_classifier = KerasClassifier(build_fn=create_lstm_model, verbose=0)

# Get the available parameters
params = lstm_classifier.get_params().keys()
print("Available parameters:", params)

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier

# Function to create LSTM model
def create_lstm_model( optimizer='adam'):
    model = Sequential()
    model.add(LSTM(units=100, input_shape=(X_train.shape[0], X_train.shape[1])))
    model.add(Dropout(0.2))  # Add dropout layer directly
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Create KerasClassifier for use in scikit-learn GridSearchCV
lstm_classifier = KerasClassifier(build_fn=create_lstm_model, verbose=0)

# Define hyperparameters to tune
param_grid = {
    'optimizer': ['adam', 'rmsprop']
}

# Perform grid search
grid_search = GridSearchCV(estimator=lstm_classifier, param_grid=param_grid, cv=3)
grid_search_result = grid_search.fit(X_train, Y_train)

# Print best parameters and best score
print("Best Parameters: ", grid_search_result.best_params_)
print("Best Score: ", grid_search_result.best_score_)


# Idiot GridSearch

In [ ]:
from sklearn.base import BaseEstimator
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.optimizers import Adam
from keras.models import Sequential
from sklearn.metrics import fbeta_score, make_scorer
from keras.layers import Embedding, Dense, SpatialDropout1D, Bidirectional, LSTM
import tensorflow.keras.backend as K

# Define your data-specific constants
MAX_WORDS = 3500
EMBEDDING_DIM = 100
MAX_SEQUENCE_LENGTH = 132

# Function to create model
def create_model(dropout_rate=0.1, LSTM_units=100, learning_rate=0.001):
    model = Sequential()
    model.add(Embedding(MAX_WORDS, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
    #model.add(Embedding(MAX_WORDS, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH)) #keras embedding
    model.add(Bidirectional(LSTM(128, return_sequences=True)))
    model.add(Dropout(0.1)) 
    model.add(Bidirectional(LSTM(64)))
    model.add(Dropout(0.1)) 
    model.add(Dense(1, activation='sigmoid'))  # linear for svm
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) #binary_crossentropy, hinge for svm [f1_m]
    model.summary()
    return model

class CustomKerasClassifier(BaseEstimator):
    def __init__(self, dropout_rate=0.1, LSTM_units=100, learning_rate=0.001, **kwargs):
        self.dropout_rate = dropout_rate
        self.LSTM_units = LSTM_units
        self.learning_rate = learning_rate
        self.kwargs = kwargs

    def fit(self, X, Y):
        self.model = create_model(dropout_rate=self.dropout_rate, LSTM_units=self.LSTM_units,
                                  learning_rate=self.learning_rate, **self.kwargs)
        self.model.fit(X, Y, **self.kwargs)
        return self

    def predict(self, X):
        return self.model.predict(X)

# Create CustomKerasClassifier instance
model = CustomKerasClassifier()


# Define the grid search parameters
param_grid = {
    'dropout_rate': [0.1,0.08,0.9],
    'LSTM_units': [50,100,150],  # Uncomment if you want to search over LSTM_units
    'learning_rate': [0.001,0.0009,0.0008]  # Uncomment if you want to search over learning_rate
}

f1 = make_scorer(fbeta_score, beta = 1)
# Perform grid search
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring = f1 , cv=3, verbose=1 , error_score="raise")  # Change scoring to 'accuracy' or 'f1' as appropriate scoring=f1
grid_result = grid_search.fit(X_train, Y_train)

# Print results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


In [ ]:
from sklearn.base import BaseEstimator
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Embedding, Dense, SpatialDropout1D, Bidirectional, LSTM
import tensorflow.keras.backend as K
# Define your data-specific constants
MAX_WORDS = 3500
EMBEDDING_DIM = 100
MAX_SEQUENCE_LENGTH = 132

# Function to create model
def create_model(dropout_rate=0.1, LSTM_units=100, learning_rate=0.001):
    model = Sequential()
    model.add(Embedding(MAX_WORDS, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
    model.add(Dense(64, activation='relu'))
    model.add(Bidirectional(LSTM(units=LSTM_units, dropout=dropout_rate, recurrent_dropout=0.1)))
    model.add(Dense(1, activation='sigmoid'))
    optimizer = Adam(learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])  
    return model

class CustomKerasClassifier(BaseEstimator):
    def __init__(self, dropout_rate=0.1, LSTM_units=100, learning_rate=0.001, **kwargs):
        self.dropout_rate = dropout_rate
        self.LSTM_units = LSTM_units
        self.learning_rate = learning_rate
        self.kwargs = kwargs

    def fit(self, X, Y):
        self.model = create_model(dropout_rate=self.dropout_rate, LSTM_units=self.LSTM_units,
                                  learning_rate=self.learning_rate, **self.kwargs)
        self.model.fit(X, Y, **self.kwargs)
        return self

    def predict(self, X):
        return self.model.predict(X)

# Create CustomKerasClassifier instance
model = CustomKerasClassifier()

# Define the grid search parameters
param_grid = {
    'dropout_rate': [0.1, 0.08, 0.09],
    'LSTM_units': [50, 100, 150],  # Uncomment if you want to search over LSTM_units
    'learning_rate': [0.001, 0.0009, 0.0008]  # Uncomment if you want to search over learning_rate
}

# Perform grid search
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, verbose=1)  # Change scoring to 'accuracy'
grid_result = grid_search.fit(X_train, Y_train)

# Print results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

# Tuna mayo

In [ ]:
import optuna 
from sklearn.base import BaseEstimator
from sklearn.model_selection import cross_val_score
from keras.optimizers import Adam
from keras.models import Sequential
from sklearn.metrics import fbeta_score, make_scorer
from keras.layers import Embedding, Dense, SpatialDropout1D, Bidirectional, LSTM
import tensorflow.keras.backend as K

# Define your data-specific constants
MAX_WORDS = 3500
EMBEDDING_DIM = 100
MAX_SEQUENCE_LENGTH = 132

# Function to create model
def create_model(trial):
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.9)
    LSTM_units = trial.suggest_categorical('LSTM_units', [50, 100, 150])
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
    
    model = Sequential()
    model.add(Embedding(MAX_WORDS, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
    model.add(Bidirectional(LSTM(LSTM_units, return_sequences=True)))
    model.add(SpatialDropout1D(dropout_rate,batch_size)) 
    model.add(Bidirectional(LSTM(int(LSTM_units / 2))))
    model.add(SpatialDropout1D(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate), metrics=['accuracy'])
    return model

class CustomKerasClassifier(BaseEstimator):
    def __init__(self, **kwargs):
        self.kwargs = kwargs

    def fit(self, X, Y):
        self.model = create_model(self.kwargs['trial'])
        self.model.fit(X, Y, **self.kwargs)
        return self

    def predict(self, X):
        return self.model.predict(X)

# Objective function for Optuna
def objective(trial):
    model = CustomKerasClassifier()
    f1 = make_scorer(fbeta_score, beta=1)
    return -np.mean(cross_val_score(model, X_train, Y_train, cv=3, scoring=f1))

# Study configuration
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Print results
print("Best trial:")
best_trial = study.best_trial
print("  Value: ", best_trial.value)
print("  Params: ")
for key, value in best_trial.params.items():
    print("    {}: {}".format(key, value))


In [24]:
import optuna
from sklearn.base import BaseEstimator
from sklearn.model_selection import cross_val_score
from keras.optimizers import Adam
from keras.models import Sequential
from sklearn.metrics import fbeta_score, make_scorer
from keras.layers import Embedding, Dense, SpatialDropout1D, Bidirectional, LSTM
import tensorflow.keras.backend as K


# Define your data-specific constants
MAX_WORDS = 3500
#EMBEDDING_DIM = 100
MAX_SEQUENCE_LENGTH = 132


def create_model(trial):
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.05, 0.3) #value should be from small to large only
    dropout_rate2 = trial.suggest_uniform('dropout_rate2', 0.05, 0.1) 
    LSTM_units = trial.suggest_categorical('LSTM_units', [64, 128, 256])
    EMBEDDING_DIM = trial.suggest_categorical('EMBEDDING_DIM', [100, 150, 200])
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-6, 1e-3)
    
    model = Sequential()
    model.add(Embedding(MAX_WORDS, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
    model.add(Bidirectional(LSTM(LSTM_units, dropout = dropout_rate ,return_sequences=True)))
    model.add(Dropout(dropout_rate2)) 
    model.add(Bidirectional(LSTM(int(LSTM_units / 2))))
    model.add(Dropout(dropout_rate2))
    model.add(Dense(1, activation='sigmoid'))
    #model.add(KerasClassifier(build_fn=create_svc_model, verbose=0))
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate), metrics=['accuracy'])
    return model


def create_model(dropout_rate, dropout_rate2, LSTM_units, EMBEDDING_DIM, learning_rate):  
    model = Sequential()
    model.add(Embedding(MAX_WORDS, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
    model.add(Bidirectional(LSTM(LSTM_units, dropout = dropout_rate ,return_sequences=True)))
    model.add(Dropout(dropout_rate2)) 
    model.add(Bidirectional(LSTM(int(LSTM_units / 2))))
    model.add(Dropout(dropout_rate2))
    model.add(Dense(1, activation='sigmoid'))
    #model.add(KerasClassifier(build_fn=create_svc_model, verbose=0))
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate), metrics=['accuracy'])
    return model

class CustomKerasClassifier( ):
    def __init__(self, trial=None, **kwargs):
        self.trial = trial
        self.kwargs = kwargs

    def fit(self, X, Y):
        self.model = create_model(self.trial)
        self.model.fit(X, Y, **self.kwargs)
        return self

    def predict(self, X):
        #adjusting array from [[0.55],[0.48],...] from first will return
        a = self.model.predict(X)
        b = [[1 if value[0] > 0.5 else 0] for value in a]
        return b

# Objective function for Optuna
def objective(trial):
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.05, 0.3) #value should be from small to large only
    dropout_rate2 = trial.suggest_uniform('dropout_rate2', 0.05, 0.1) 
    LSTM_units = trial.suggest_categorical('LSTM_units', [64, 128, 256])
    EMBEDDING_DIM = trial.suggest_categorical('EMBEDDING_DIM', [100, 150, 200])
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-6, 1e-3)
    model = CustomKerasClassifier(trial=trial)
    f1 = make_scorer(fbeta_score, beta=1)
    print('result score f1 fuc', -np.mean(cross_val_score(model, X_train, Y_train, cv=3, scoring=f1)))
    return np.mean(cross_val_score(model, X_train, Y_train, cv=3, scoring=f1))

def early_stop(study, trial):
    if study.best_value is None:
        return False
    return study.best_value >= -trial.value


# Study configuration
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=1, callbacks=[early_stop]) #edit the running here

# Print results
print("Best trial:")
best_trial = study.best_trial
print("  Value: ", best_trial.value)
print("  Params: ", study.best_params)
# for key, value in best_trial.params.items():
#     print("    {}: {}".format(key, value))


c:\Users\mrbvb\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2024-04-19 17:50:20,342] A new study created in memory with name: no-name-c4265abb-0db2-437e-9b5e-5f543e62c817
C:\Users\mrbvb\AppData\Local\Temp\ipykernel_32876\2772981875.py:53: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_rate = trial.suggest_uniform('dropout_rate', 0.05, 0.3) #value should be from small to large only
C:\Users\mrbvb\AppData\Local\Temp\ipykernel_32876\2772981875.py:54: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float

9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 200ms/step - accuracy: 0.5163 - loss: 0.6938
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 146ms/step


C:\Users\mrbvb\AppData\Local\Temp\ipykernel_32876\2772981875.py:18: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_rate = trial.suggest_uniform('dropout_rate', 0.05, 0.3) #value should be from small to large only
C:\Users\mrbvb\AppData\Local\Temp\ipykernel_32876\2772981875.py:19: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_rate2 = trial.suggest_uniform('dropout_rate2', 0.05, 0.1)
C:\Users\mrbvb\AppData\Local\Temp\ipykernel_32876\2772981875.py:22: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_logunifor

10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 199ms/step - accuracy: 0.5186 - loss: 0.6947
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 141ms/step


C:\Users\mrbvb\AppData\Local\Temp\ipykernel_32876\2772981875.py:18: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_rate = trial.suggest_uniform('dropout_rate', 0.05, 0.3) #value should be from small to large only
C:\Users\mrbvb\AppData\Local\Temp\ipykernel_32876\2772981875.py:19: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_rate2 = trial.suggest_uniform('dropout_rate2', 0.05, 0.1)
C:\Users\mrbvb\AppData\Local\Temp\ipykernel_32876\2772981875.py:22: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_logunifor

...

10/10 ━━━━━━━━━━━━━━━━━━━━ 7s 235ms/step - accuracy: 0.4650 - loss: 0.6946
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 151ms/step


C:\Users\mrbvb\AppData\Local\Temp\ipykernel_32876\2772981875.py:18: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_rate = trial.suggest_uniform('dropout_rate', 0.05, 0.3) #value should be from small to large only
C:\Users\mrbvb\AppData\Local\Temp\ipykernel_32876\2772981875.py:19: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_rate2 = trial.suggest_uniform('dropout_rate2', 0.05, 0.1)
C:\Users\mrbvb\AppData\Local\Temp\ipykernel_32876\2772981875.py:22: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_logunifor

10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 204ms/step - accuracy: 0.5408 - loss: 0.6938
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 206ms/step


[I 2024-04-19 17:51:04,517] Trial 0 finished with value: 0.20000000000000004 and parameters: {'dropout_rate': 0.28499632963784055, 'dropout_rate2': 0.07985606671874927, 'LSTM_units': 256, 'EMBEDDING_DIM': 200, 'learning_rate': 0.0003522637703900681}. Best is trial 0 with value: 0.20000000000000004.


Best trial:
  Value:  0.20000000000000004
  Params:  {'dropout_rate': 0.28499632963784055, 'dropout_rate2': 0.07985606671874927, 'LSTM_units': 256, 'EMBEDDING_DIM': 200, 'learning_rate': 0.0003522637703900681}


In [ ]:
optuna.visualization.plot_optimization_history(study)
optuna.visualization.plot_slice(study)
optuna.visualization.plot_contour(study, params=['dropout_rate','dropout_rate2','LSTM_units','EMBEDDING_DIM','learning_rate'])


In [36]:
best_param = study.best_params

In [29]:
import matplotlib.pyplot as plt

optuna.visualization.plot_optimization_history(study)

In [30]:
optuna.visualization.plot_parallel_coordinate(study)

In [32]:
optuna.visualization.plot_slice(study,params=['dropout_rate','dropout_rate2','LSTM_units','EMBEDDING_DIM','learning_rate'])

In [33]:
optuna.visualization.plot_param_importances(study)

ValueError: Cannot evaluate parameter importances with only a single trial.

In [37]:
best_param

{'dropout_rate': 0.28499632963784055,
 'dropout_rate2': 0.07985606671874927,
 'LSTM_units': 256,
 'EMBEDDING_DIM': 200,
 'learning_rate': 0.0003522637703900681}

In [38]:
# 'dropout_rate','dropout_rate2','LSTM_units','EMBEDDING_DIM','learning_rate
best_dropout_1 = best_param['dropout_rate']
best_dropout_2 = best_param['dropout_rate2']
best_LSTM_units = best_param['LSTM_units']
best_EMBEDDING_DIM = best_param['EMBEDDING_DIM']
best_learning_rate = best_param['learning_rate']

In [ ]:
best_model = CustomKerasClassifier(trial=n)

In [ ]:
def create_model(dropout_rate, dropout_rate2, LSTM_units, EMBEDDING_DIM, learning_rate):  
    model = Sequential()
    model.add(Embedding(MAX_WORDS, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
    model.add(Bidirectional(LSTM(LSTM_units, dropout = dropout_rate ,return_sequences=True)))
    model.add(Dropout(dropout_rate2)) 
    model.add(Bidirectional(LSTM(int(LSTM_units / 2))))
    model.add(Dropout(dropout_rate2))
    model.add(Dense(1, activation='sigmoid'))
    #model.add(KerasClassifier(build_fn=create_svc_model, verbose=0))
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate), metrics=['accuracy'])
    return model